In [1]:
# !pip install pyshp
# !pip install reverse-geocode
# !pip install reverse_geocoder

# import reverse_geocode
import reverse_geocoder as rg
import pandas as pd

def read_shapefile(shp_path):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' column holding
    the geometry information. This uses the pyshp package
    """
    import shapefile

    #read file, parse out the records and shapes
    sf = shapefile.Reader(shp_path)
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]

    #write into a dataframe
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)

    return df

In [2]:
# Shape path
shp_path = "../data/outdoor_inventory/Outdoor_Inventory_AV.shp"

df = read_shapefile(shp_path)
df

,PanelID,X,Y,Max_Visibi,Average_Da,coords
0,26144,-8.473975,40.900768,69,31.0,"[[-8.473975, 40.900768]]"
1,11714,-9.315949,38.958125,69,31.0,"[[-9.315949, 38.958125]]"
2,26109,-8.510079,40.871821,69,32.0,"[[-8.510079, 40.871821]]"
3,32555,-8.479751,40.837695,69,32.0,"[[-8.479751, 40.837695]]"
4,5291,-8.429870,40.268692,69,32.0,"[[-8.42987, 40.268692]]"
...,...,...,...,...,...,...
30370,4671,-9.415045,38.701264,145,71045.0,"[[-9.415045, 38.701264]]"
30371,4676,-9.415113,38.701244,145,73035.0,"[[-9.415113, 38.701244]]"
30372,32007,-9.171094,38.703705,180,76575.0,"[[-9.171094, 38.703705]]"
30373,29984,-9.165918,38.750425,180,79714.0,"[[-9.165918, 38.750425]]"


In [9]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="google")


In [22]:
lst = []

for i, row in df[0:20].iterrows():

    reverse_loc = locator.reverse((row['Y'], row['X'])).raw
    lst.append(
        {
            'original_X': row['X'],
            'original_Y': row['Y'],
            'address': reverse_loc['address'].get('road')
        }
    )

In [23]:
pd.DataFrame.from_records(lst)

,original_X,original_Y,address
0,-8.473975,40.900768,Rua Alto das Casas
1,-9.315949,38.958125,EN 9
2,-8.510079,40.871821,Rua Professor Doutor António Joaquim Ferreira ...
3,-8.479751,40.837695,Rua Aníbal Beleza
4,-8.429870,40.268692,Rua da Escola
5,-8.414758,41.045215,Largo do Ripado
6,-8.749668,41.368876,EN 206
7,-9.146613,38.687800,Largo Alfredo Diniz (Alex)
8,-8.655476,41.200398,Rua General José Vicente de Freitas
9,-9.143221,38.710728,Praça Luís de Camões


In [18]:
locator.reverse((38.75665, -9.290662)).raw

{'place_id': 153694071,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 256787284,
 'lat': '38.756532135128595',
 'lon': '-9.290802163630863',
 'display_name': 'IC 19;A 37, Massamá, Massamá e Monte Abraão, Sintra, Lisboa, Grande Lisboa, Área Metropolitana de Lisboa, 2745-862, Portugal',
 'address': {'road': 'IC 19;A 37',
  'suburb': 'Massamá',
  'city_district': 'Massamá',
  'city': 'Massamá e Monte Abraão',
  'municipality': 'Sintra',
  'county': 'Lisboa',
  'state_district': 'Grande Lisboa',
  'state': 'Área Metropolitana de Lisboa',
  'postcode': '2745-862',
  'country': 'Portugal',
  'country_code': 'pt'},
 'boundingbox': ['38.7520703', '38.756785', '-9.2910148', '-9.2880515']}

In [14]:
lst[0]

Location(Rua Alto das Casas, Macieira de Sarnes, São João da Madeira, Aveiro, Área Metropolitana do Porto, Norte, 3700-164, Portugal, (40.90072373526348, -8.473955783546577, 0.0))

In [6]:
tmp["freguesia"] = tmp.apply(lambda x: rg.search((x["Y"], x["X"]))[0]['name'], axis=1)
tmp["concelho"] = tmp.apply(lambda x: rg.search((x["Y"], x["X"]))[0]['admin2'], axis=1)
tmp["distrito"] = tmp.apply(lambda x: rg.search((x["Y"], x["X"]))[0]['admin1'], axis=1)

tmp.to_csv("../data/df_reverse_coords.csv")
tmp

Loading formatted geocoded file...


,PanelID,X,Y,Max_Visibi,Average_Da,freguesia,concelho,distrito
0,3,-8.501357,40.126293,180,3032.0,Condeixa-a-Nova,Condeixa-A-Nova,Coimbra
1,4,-9.103966,38.808856,180,22893.0,Bobadela,Loures,Lisbon
2,5,-8.622777,41.241615,180,20946.0,Maia,Maia,Porto
3,6,-9.166976,38.647052,180,8443.0,Corroios,Seixal,Setubal
4,7,-8.501363,40.126291,180,2264.0,Condeixa-a-Nova,Condeixa-A-Nova,Coimbra
...,...,...,...,...,...,...,...,...
30370,49322,-9.167794,38.723143,88,17692.0,Lisbon,Lisbon,Lisbon
30371,49323,-9.165695,38.765418,88,5542.0,Olival do Basto,Odivelas,Lisbon
30372,49324,-9.165794,38.765409,71,1836.0,Olival do Basto,Odivelas,Lisbon
30373,49325,-8.640424,41.183925,88,12111.0,Senhora da Hora,Matosinhos,Porto
